# Tutorial 1: Getting Started with Marine Engineering Module

**Duration:** ~15 minutes  
**Level:** Beginner  
**Prerequisites:** Basic Python knowledge

## Learning Objectives

By the end of this tutorial, you will be able to:
1. Install and import the marine engineering modules
2. Run your first catenary calculation
3. Generate a wave spectrum
4. Create professional visualizations
5. Understand the module structure

## Table of Contents

1. [Installation & Setup](#installation)
2. [First Catenary Calculation](#catenary)
3. [Wave Spectrum Generation](#waves)
4. [Visualization](#visualization)
5. [Next Steps](#next-steps)

## 1. Installation & Setup <a id="installation"></a>

First, let's ensure all required packages are installed:

In [ ]:
# Install required packages (uncomment if needed)
# !pip install numpy scipy matplotlib pandas plotly ipywidgets

# Import core libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

# Configure matplotlib for inline plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Core libraries imported successfully")

In [ ]:
# Import marine engineering modules
import sys
sys.path.append(str(Path.cwd().parent.parent / 'src'))

from marine_engineering.catenary.solver import CatenarySolver, CatenaryInput
from marine_engineering.wave_spectra.spectra import JONSWAPSpectrum, PiersonMoskowitzSpectrum, WaveSpectrumParameters

print("✓ Marine engineering modules imported successfully")
print("\n📦 Available modules:")
print("  - Catenary Solver: Mooring line analysis")
print("  - Wave Spectra: JONSWAP and Pierson-Moskowitz")
print("  - Mooring Analysis: Component database and design")
print("  - Environmental Loading: OCIMF wind/current forces")

## 2. First Catenary Calculation <a id="catenary"></a>

Let's calculate the shape and tension of a mooring line using the catenary solver.

### Problem Setup

We have a mooring line with:
- Length: 500 meters
- Horizontal span (anchor to fairlead): 400 meters  
- Vertical span (fairlead above anchor): 50 meters
- Weight per length: 1200 N/m (in water)
- Axial stiffness (EA): 800,000 kN

In [ ]:
# Define mooring line parameters
params = CatenaryInput(
    length=500.0,           # Total line length [m]
    horizontal_span=400.0,  # Horizontal distance [m]
    vertical_span=50.0,     # Vertical distance [m]
    weight_per_length=1200.0,  # Submerged weight [N/m]
    ea_stiffness=800_000_000.0,  # Axial stiffness [N]
    water_depth=100.0       # Water depth for touchdown calculation [m]
)

# Create solver instance
solver = CatenarySolver(tolerance=1e-6, max_iterations=200)

# Solve the catenary problem
print("⚙️  Solving catenary boundary value problem...")
results = solver.solve(params)
print("✓ Solution converged!\n")

# Display results
print("="*60)
print("CATENARY ANALYSIS RESULTS")
print("="*60)
print(f"Horizontal Tension (H):        {results.horizontal_tension/1000:.1f} kN")
print(f"Vertical Tension (Fairlead):   {results.vertical_tension_fairlead/1000:.1f} kN")
print(f"Total Tension (Fairlead):      {results.total_tension_fairlead/1000:.1f} kN")
print(f"Total Tension (Anchor):        {results.total_tension_anchor/1000:.1f} kN")
print(f"Line Elongation:               {results.elongation:.2f} m")
print(f"Catenary Parameter (a):        {results.catenary_parameter:.1f} m")
if results.touchdown_distance:
    print(f"Touchdown Distance:            {results.touchdown_distance:.1f} m")
print(f"Converged:                     {results.converged}")
print(f"Iterations:                    {results.iterations}")
print("="*60)

In [ ]:
# Visualize the catenary shape
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Catenary shape
ax1.plot(results.shape_x, results.shape_y, 'b-', linewidth=2, label='Mooring Line')
ax1.plot(0, 0, 'go', markersize=12, label='Anchor', zorder=5)
ax1.plot(params.horizontal_span, params.vertical_span, 'ro', markersize=12, label='Fairlead', zorder=5)

# Add seabed
if params.water_depth:
    ax1.axhline(y=-params.water_depth, color='brown', linestyle='--', linewidth=1.5, label='Seabed', alpha=0.7)

ax1.grid(True, alpha=0.3)
ax1.set_xlabel('Horizontal Distance [m]', fontsize=11)
ax1.set_ylabel('Vertical Distance [m]', fontsize=11)
ax1.set_title('Mooring Line Catenary Shape', fontsize=12, fontweight='bold')
ax1.legend(loc='best', fontsize=10)
ax1.set_aspect('equal', adjustable='datalim')

# Plot 2: Tension distribution
ax2.plot(results.shape_x, results.tension_distribution/1000, 'r-', linewidth=2)
ax2.axhline(y=results.horizontal_tension/1000, color='blue', linestyle='--', 
            linewidth=1.5, label=f'H = {results.horizontal_tension/1000:.1f} kN', alpha=0.7)
ax2.grid(True, alpha=0.3)
ax2.set_xlabel('Horizontal Distance [m]', fontsize=11)
ax2.set_ylabel('Tension [kN]', fontsize=11)
ax2.set_title('Tension Distribution Along Line', fontsize=12, fontweight='bold')
ax2.legend(loc='best', fontsize=10)

plt.tight_layout()
plt.savefig('../output/tutorial1_catenary.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Catenary visualization complete")

## 3. Wave Spectrum Generation <a id="waves"></a>

Now let's generate a JONSWAP wave spectrum for a typical offshore condition.

In [ ]:
# Define wave conditions
wave_params = WaveSpectrumParameters(
    Hs=4.0,              # Significant wave height [m]
    Tp=10.0,             # Peak period [s]
    gamma=3.3,           # JONSWAP peak enhancement factor
    freq_range=(0.01, 0.5),  # Frequency range [Hz]
    n_frequencies=200    # Number of frequency bins
)

# Create JONSWAP spectrum
jonswap = JONSWAPSpectrum(wave_params)
spectrum = jonswap.compute_spectrum()

# Calculate spectral statistics
stats = jonswap.get_spectral_statistics()

print("="*60)
print("WAVE SPECTRUM ANALYSIS")
print("="*60)
print(f"Input Hs:                {wave_params.Hs:.2f} m")
print(f"Calculated Hs:           {stats['Hs']:.2f} m")
print(f"Peak Period (Tp):        {wave_params.Tp:.2f} s")
print(f"Zero-Crossing Period:    {stats['Tz']:.2f} s")
print(f"Mean Period:             {stats['Tm']:.2f} s")
print(f"Spectral Bandwidth:      {stats['bandwidth']:.3f}")
print(f"\nSpectral Moments:")
print(f"  m₀: {stats['m0']:.4f}")
print(f"  m₁: {stats['m1']:.4f}")
print(f"  m₂: {stats['m2']:.4f}")
print(f"  m₄: {stats['m4']:.4f}")
print("="*60)

## 4. Visualization <a id="visualization"></a>

Let's create professional visualizations of the wave spectrum.

In [ ]:
# Create comprehensive wave spectrum plot
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Spectral density vs frequency
ax1 = axes[0, 0]
ax1.plot(jonswap.frequencies, spectrum, 'b-', linewidth=2)
ax1.fill_between(jonswap.frequencies, spectrum, alpha=0.3)
ax1.axvline(x=1/wave_params.Tp, color='r', linestyle='--', linewidth=1.5, 
            label=f'Peak freq = {1/wave_params.Tp:.3f} Hz')
ax1.set_xlabel('Frequency [Hz]', fontsize=11)
ax1.set_ylabel('Spectral Density S(f) [m²s]', fontsize=11)
ax1.set_title('JONSWAP Wave Spectrum', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=10)

# Plot 2: Spectral density vs period
ax2 = axes[0, 1]
periods = 1 / jonswap.frequencies[1:]  # Avoid division by zero
spectrum_T = spectrum[1:] / (jonswap.frequencies[1:]**2)  # Convert to period-based
ax2.plot(periods, spectrum_T, 'g-', linewidth=2)
ax2.fill_between(periods, spectrum_T, alpha=0.3, color='green')
ax2.axvline(x=wave_params.Tp, color='r', linestyle='--', linewidth=1.5, 
            label=f'Tp = {wave_params.Tp:.1f} s')
ax2.set_xlabel('Period [s]', fontsize=11)
ax2.set_ylabel('Spectral Density S(T) [m²/s]', fontsize=11)
ax2.set_title('Spectrum in Period Domain', fontsize=12, fontweight='bold')
ax2.set_xlim(0, 25)
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=10)

# Plot 3: Cumulative energy
ax3 = axes[1, 0]
cumulative_energy = np.cumsum(spectrum) * (jonswap.frequencies[1] - jonswap.frequencies[0])
normalized_energy = cumulative_energy / cumulative_energy[-1] * 100
ax3.plot(jonswap.frequencies, normalized_energy, 'purple', linewidth=2)
ax3.axhline(y=50, color='r', linestyle='--', linewidth=1, alpha=0.5)
ax3.axhline(y=90, color='orange', linestyle='--', linewidth=1, alpha=0.5)
ax3.set_xlabel('Frequency [Hz]', fontsize=11)
ax3.set_ylabel('Cumulative Energy [%]', fontsize=11)
ax3.set_title('Cumulative Energy Distribution', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.set_ylim(0, 100)

# Plot 4: Statistics summary (text box)
ax4 = axes[1, 1]
ax4.axis('off')
stats_text = f"""
WAVE SPECTRUM STATISTICS
{'='*40}

Sea State Parameters:
  Significant Height (Hs):  {stats['Hs']:.2f} m
  Peak Period (Tp):         {wave_params.Tp:.2f} s
  Zero-Crossing Period:     {stats['Tz']:.2f} s
  Mean Period:              {stats['Tm']:.2f} s

Spectral Characteristics:
  Peak Enhancement (γ):     {wave_params.gamma:.1f}
  Spectral Bandwidth (ε):   {stats['bandwidth']:.3f}
  
Spectral Moments:
  m₀ (variance):            {stats['m0']:.4f} m²
  m₁:                       {stats['m1']:.4f} m²/s
  m₂:                       {stats['m2']:.4f} m²/s²
  m₄:                       {stats['m4']:.6f} m²/s⁴

Frequency Range:            {wave_params.freq_range[0]:.3f} - {wave_params.freq_range[1]:.3f} Hz
Number of Frequencies:      {wave_params.n_frequencies}
"""
ax4.text(0.1, 0.5, stats_text, fontsize=11, family='monospace',
         verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

plt.suptitle('JONSWAP Wave Spectrum Analysis - Tutorial 1', fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('../output/tutorial1_wave_spectrum.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Wave spectrum visualization complete")

## 5. Quick Example: Comparing JONSWAP and Pierson-Moskowitz <a id="comparison"></a>

In [ ]:
# Create Pierson-Moskowitz spectrum with same parameters
pm = PiersonMoskowitzSpectrum(wave_params)
spectrum_pm = pm.compute_spectrum()

# Compare the two spectra
plt.figure(figsize=(12, 6))
plt.plot(jonswap.frequencies, spectrum, 'b-', linewidth=2, label='JONSWAP (γ=3.3)')
plt.plot(pm.frequencies, spectrum_pm, 'r--', linewidth=2, label='Pierson-Moskowitz (γ=1.0)')
plt.fill_between(jonswap.frequencies, spectrum, alpha=0.2, color='blue')
plt.fill_between(pm.frequencies, spectrum_pm, alpha=0.2, color='red')
plt.axvline(x=1/wave_params.Tp, color='green', linestyle='--', linewidth=1.5, 
            label=f'Peak freq = {1/wave_params.Tp:.3f} Hz', alpha=0.7)
plt.xlabel('Frequency [Hz]', fontsize=12)
plt.ylabel('Spectral Density S(f) [m²s]', fontsize=12)
plt.title('JONSWAP vs Pierson-Moskowitz Spectra Comparison\nHs=4m, Tp=10s', fontsize=13, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../output/tutorial1_spectrum_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 Key Differences:")
print(f"  - JONSWAP has sharper peak (γ={wave_params.gamma})")
print(f"  - P-M represents fully developed seas (γ=1.0)")
print(f"  - JONSWAP is used for fetch-limited conditions")
print(f"  - Both produce same Hs: {jonswap.significant_wave_height():.2f} m")

## 6. Summary & Next Steps <a id="next-steps"></a>

### What You've Learned

✅ **Installation & Setup**
- Imported marine engineering modules
- Configured visualization environment

✅ **Catenary Analysis**
- Solved mooring line catenary problem
- Calculated tensions and elongation
- Visualized line shape and tension distribution

✅ **Wave Spectra**
- Generated JONSWAP and P-M spectra
- Calculated spectral moments and statistics
- Created professional visualizations

### Next Tutorials

**Tutorial 2: Catenary Analysis Deep Dive**
- Simplified vs BVP solver comparison
- Parameter sensitivity analysis
- Multi-segment lazy-wave risers
- Real FPSO case study

**Tutorial 3: Wave Spectra Mastery**
- JONSWAP parameter selection
- Spectral moments interpretation
- Irregular wave synthesis
- Integration with motion analysis

**Tutorial 4: Mooring System Design**
- Component database exploration
- 8-point spread mooring layout
- Tension analysis workflow
- Safety factor calculations

### Practice Exercises

Try modifying the examples above:

1. **Catenary Exercise**: Change the line parameters and observe how tension changes
2. **Wave Exercise**: Try different Hs and Tp values for various sea states
3. **Comparison Exercise**: Vary gamma from 1.0 to 5.0 and see the effect

### Resources

- API RP 2SK: Mooring design guidelines
- DNV-RP-C205: Environmental conditions
- OCIMF MEG4: Mooring equipment guidelines

In [ ]:
# Save session results for next tutorial
import pickle

session_data = {
    'catenary_results': results,
    'wave_spectrum': spectrum,
    'wave_params': wave_params,
    'spectral_stats': stats
}

with open('../data/tutorial1_session.pkl', 'wb') as f:
    pickle.dump(session_data, f)

print("✓ Session data saved to ../data/tutorial1_session.pkl")
print("\n🎉 Tutorial 1 Complete! Ready for Tutorial 2.")